In [16]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain")

In [17]:
import os
import openai
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


openai.api_key = os.environ.get("MY_OPENAI_API_KEY")

In [18]:
import json

def process_json_data(json_files):
    """
    여러 JSON 파일을 읽고 데이터를 통합한 후 특정 형식의 문자열 리스트로 반환

    Parameters:
        json_files (list): JSON 파일 경로 리스트

    Returns:
        list: 파일 데이터에서 'title'과 'content'를 읽어 특정 형식으로 변환한 리스트
    """
    all_json_data = []
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                all_json_data.extend(data)
        except FileNotFoundError:
            print(f"Error: 파일을 찾을 수 없습니다 - {file_path}")
        except json.JSONDecodeError:
            print(f"Error: JSON 파일 형식이 잘못되었습니다 - {file_path}")

    # 'title'과 'content'를 읽어 특정 형식으로 반환
    return [
        f"Title: {item.get('title', 'N/A')}\nContent: {item.get('content', 'N/A')}"
        for item in all_json_data
    ]

In [19]:
json_files = ['./documents/filtered_unsolved_cases.json', './documents/korea_crime.json']
crimes = process_json_data(json_files)

In [20]:
from pprint import pprint
sample = crimes[0]
pprint(sample)

('Title: 정인숙 피살사건\n'
 'Content: 정인숙 피살사건(鄭仁淑 被殺事件)은제3공화국당시인1970년3월 17일밤 11시경,대한민국서울특별시마포구합정동절두산근처 '
 '도로에서 발생한 교통사고를 가장한 총격 살인사건으로 고급 요정 종업원인정인숙이 사망한 사건이다.[1][2]피해자 정인숙(본명: 정금지)은 '
 '총상으로 요절하였고 그의 차를 운전하던 넷째 오빠 정종욱은 넓적다리에 관통상을 입었으나 지나가던 택시기사에게 도움을 청하여 구조되었다.\n'
 '정인숙은 당시 출산한 아이의 아버지인대한민국의 국무총리정일권과 갈등관계에 있었고,신민당은 이 사건의 배후로 정부 고위층의 개입 의혹을 '
 '제기했으나 유야무야 묻혀져 아직까지의문사로 남아있다.\n'
 '경과[편집]\n'
 '경과\n'
 '1970년3월 17일밤 11시경, 서울마포구합정동절두산 부근의강변3로에 멈춰 서 있는검은색 코로나 승용차에서 권총에 넓적다리를 관통당해 '
 '신음하고 있는 한 사내와, 머리와 가슴에 총을 맞아 이미 숨진 한 젊은 여인이 발견되었다. 부상당한 사내는정종욱(당시 34세), 숨진 '
 '여인은 정인숙(당시 25세)으로 두 사람은 남매 관계로 밝혀졌다.\n'
 '당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다.[3][4]아이의 아버지에 대해서는 당시 정부의 한 유력인사가 '
 '지목되기도 했다.\n'
 '고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 정인숙은 당시한일회담도 이루어진 선운각 등 최고급 '
 '요정에서 호스티스로 일하고 있었다.[4]\n'
 '나중에 정인숙의 집에서 발견된 정인숙의 소지품에선 정관계 고위층의 명함 26장이 포함된 33장의 명함이 쏟아져 나왔는데, '
 '명단에는박정희,정일권,이후락,김형욱등 대다수 5.16 주체세력들이  포함되어 있었다.[5]이후 경찰 수사는 지지부진해졌고 언론 보도가 '
 '수사를 대신하게 되었다. 언론은 정인숙에게 숨겨진 아들(정일성,정성일, 또는박승일(68년

In [21]:
len(sample)

19445

### 불필요한 텍스트, 참고링크 정보, 관련 사건 정보가 다수 포함

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def split_texts(texts, chunk_size=1000, chunk_overlap=200):
    """
    텍스트 데이터를 입력 받아 분할

    Parameters:
        texts: 텍스트 데이터

    Returns:
        Document 객체
    """
    recursive_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )
    documents = [Document(page_content=texts)]
    return recursive_text_splitter.split_documents(documents)

In [23]:
splits = split_texts(sample)

### 문서를 분할 후 순차적으로 llm으로 필터링
### 입력 토큰에 여유가 있다면 그대로 사용 가능
### ConversationSummaryMemory 에 저장된 전체 맥락에 맞는 정보만 남김

In [24]:
from langchain.memory import ConversationSummaryMemory

def documents_filter(SPLITS):
    """
    분할된 데이터에서 불필요한 데이터를 제거하고 하나로 결합
    ConversationSummaryMemory에 이전 내용을 요약하여 저장
    아전 내용과 대조해서 불필요한 데이터 구분

    Parameters:
        SPLITS: 분할된 텍스트 데이터 : Document

    Returns:
        텍스트 데이터
    """
    llm = ChatOpenAI(
                model="gpt-4o-mini",
                api_key=openai.api_key,
                max_tokens=1000,
                temperature=0.0,
            )
    summaries = []
    memory = ConversationSummaryMemory(
        llm=llm, return_messages=True)
    
    count = 0
    for SPLIT in SPLITS:
        SPLIT = SPLIT.page_content

        try:
            context = memory.load_memory_variables({})["history"]
            prompt = ChatPromptTemplate.from_template(
                """
                persona : documents filter
                language : only in korean
                extract the parts related to the context and ignore the rest,
                write blanck if it's not relevant,
                
                <context>
                {context}
                </context>
                
                <docs>
                {SPLIT}
                </docs>
                """
            )
            chain = prompt | llm | StrOutputParser()
            summary = chain.invoke({"SPLIT": SPLIT, 'context' : context})
            memory.save_context({"input": f'summary # {count}'}, {"output": summary})
            summaries.append(summary)
            count+=1

        except Exception as e:
            # 오류 처리: 만약 API 호출 중에 문제가 발생하면 오류 메시지 추가
            print(f"Error summarizing document: {e}")
            summaries.append(f"Error summarizing document: {e}")

    return "".join(summaries)

In [32]:
summary = documents_filter(splits)
len(summary)

14327

In [33]:
print(summary)

<context>
[SystemMessage(content=, additional_kwargs={}, response_metadata={})]
</context>

<docs>
Title: 정인숙 피살사건
Content: 정인숙 피살사건(鄭仁淑 被殺事件)은 1970년 3월 17일 밤 11시경, 대한민국 서울특별시 마포구 합정동 절두산 근처 도로에서 발생한 교통사고를 가장한 총격 살인사건으로 고급 요정 종업원인 정인숙이 사망한 사건이다. 피해자 정인숙(본명: 정금지)은 총상으로 요절하였고 그의 차를 운전하던 넷째 오빠 정종욱은 넓적다리에 관통상을 입었으나 지나가던 택시기사에게 도움을 청하여 구조되었다. 정인숙은 당시 출산한 아이의 아버지인 대한민국의 국무총리 정일권과 갈등관계에 있었고, 신민당은 이 사건의 배후로 정부 고위층의 개입 의혹을 제기했으나 유야무야 묻혀져 아직까지 의문사로 남아있다. 1970년 3월 17일 밤 11시경, 서울 마포구 합정동 절두산 부근의 강변 3로에 멈춰 서 있는 검은색 코로나 승용차에서 권총에 넓적다리를 관통당해 신음하고 있는 한 사내와, 머리와 가슴에 총을 맞아 이미 숨진 한 젊은 여인이 발견되었다. 부상당한 사내는 정종욱(당시 34세), 숨진 여인은 정인숙(당시 25세)으로 두 사람은 남매 관계로 밝혀졌다. 당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다. 아이의 아버지에 대해서는 당시 정부의 한 유력인사가 지목되기도 했다. 고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 정인숙은 당시 한일회담도 이루어진 선운각 등 최고급 요정에서 호스티스로 일하고 있었다당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다. 아이의 아버지에 대해서는 당시 정부의 한 유력인사가 지목되기도 했다. 고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 정인숙은 당시한일회담도 이루어진 선운각 등 최고급 요정에서 호스티스로 일하고 있었다. 나중에 정

In [43]:
def generate_script(summaries):
    """
    대화 LLM이 전달할 이야기의 대본 생성

    Parameters:
        summaries: 필터링된 텍스트 데이터

    Returns:
        텍스트 데이터
    """
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key=openai.api_key,
        max_tokens=5000,
        temperature=0.0,
    )
    prompt = ChatPromptTemplate.from_template(
    """
    persona = script writer
    language = only in korean
    least 3000 tokens
    use input,
    refer to sample,
    write about time, character, event,
    write only fact
    ignore the mere listing of facts and write N/A
 
    <sample>
    # title : title of script
    # prologue 1 : song, movie, book, show about subject
    - coontent :
    # prologue 2 : explain about subject
    - coontent :
    # prologue 3 : explain about character
    - coontent :
    # exposition 1 : historical background of subject
    - coontent :
    # exposition 2 : history of character
    - coontent :
    # exposition 3 : beginning of event
    - coontent :
    # development 1 : situation, action, static of character
    - coontent :
    # development 2 : influence of event
    - coontent :
    # development 3 : reaction of people
    - coontent :
    # climax 1 : event and effect bigger
    - coontent :
    # climax 2 : dramatic action, conflict
    - coontent :
    # climax 3 : falling Action
    - coontent :
    # denouement : resolution
    - coontent :
    # epilogue : message, remaining
    - coontent :
    </sample>

    <input>
    {summaries}
    </input>

    """
    )
    chain = prompt | llm | StrOutputParser()
    script = chain.invoke({"summaries": summaries})
    return script

In [44]:
script = generate_script(summary)

In [45]:
len(script)

1790

In [46]:
print(script)

# 제목: 정인숙 피살사건

# 프롤로그 1: 사건을 다룬 미디어
- 1995년 MBC 특별기획 드라마 '제4공화국'에서 정인숙 피살사건을 주제로 다루며, 사건의 복잡성과 정치적 배경을 극화하였다. 이 드라마는 사건의 진실을 탐구하는 과정에서 시청자들에게 큰 충격과 흥미를 불러일으켰다.

# 프롤로그 2: 사건의 배경
- 정인숙 피살사건은 1970년 3월 17일 서울에서 발생한 교통사고로 위장된 살인 사건이다. 피해자인 정인숙은 고급 호스티스였으며, 그녀의 죽음은 한국 사회의 권력 구조와 정치적 음모를 드러내는 중요한 사건으로 남아 있다.

# 프롤로그 3: 정인숙의 인물상
- 정인숙은 고위 공무원의 딸로 태어나 대학을 중퇴한 후, 고급 요정에서 호스티스로 일하며 여러 정치인들과의 관계를 맺었다. 그녀는 3세 아들을 두었으며, 아들의 아버지에 대한 의혹이 사건의 중심에 있다.

# 전개 1: 사건 발생의 역사적 배경
- 1970년대 초, 한국은 군사 정권 하에 있었고, 정치적 억압과 부패가 만연했다. 정인숙의 죽음은 이러한 사회적 배경 속에서 권력자들의 비리와 음모를 드러내는 계기가 되었다.

# 전개 2: 정인숙의 개인사
- 정인숙은 여러 명의 친오빠가 있었고, 그 중 넷째 오빠인 정종욱이 사건 당시 그녀의 운전기사 역할을 하고 있었다. 정인숙은 당시 국무총리 정일권과의 갈등으로 인해 사건의 배경이 복잡해졌다.

# 전개 3: 사건의 시작
- 1970년 3월 17일 밤, 정인숙과 정종욱은 서울 마포구 합정동에서 총격을 당했다. 정인숙은 머리와 가슴에 총을 맞아 사망하였고, 정종욱은 부상을 입었다. 이 사건은 교통사고로 위장되었으나, 곧 살인 사건으로 밝혀졌다.

# 전개 4: 사건의 진행과 정종욱의 역할
- 사건 발생 후, 정종욱은 경찰에 의해 범인으로 지목되었다. 그는 정인숙의 문란한 행실을 지적하며 갈등이 있었고, 가족의 명예를 지키기 위해 그녀를 살해했다고 주장했다. 그러나 그의 주장은 의혹을 불러일으켰다.

# 전개 5: 사건의 영향
- 정인숙의 죽음

In [47]:
new_script = script
script_documents = [
                Document(page_content=new_script, metadata={"source": "script"}),
            ]

In [48]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

def load_vector_store(db_name: str, DB_PATH):
        """
        로컬에 저장된 크로마 db를 불러옴

        Parameters:
            db_name : db 생성시 설정한 이름
            DB_PATH : db 경로
        Returns:
            Chroma 객체
        """
        return Chroma(
        collection_name=db_name,
        persist_directory=DB_PATH,
        embedding_function=OpenAIEmbeddings(
            model="text-embedding-ada-002", api_key=openai.api_key
        ),
    )

In [49]:
PATH = './chatbot/db'
script_path = PATH + '/script_db'
script_db = load_vector_store('script_db', script_path)

/var/folders/s_/wv5qgx_s4vq4yfjdjfxl0x_c0000gn/T/ipykernel_19944/1668821023.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(


In [50]:
script_db.add_documents(script_documents)

['acb2fc92-c9a2-49b9-8ee3-a7b165a13f8a']

In [61]:
from langchain_core.prompts import PromptTemplate
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

openai.api_key = os.environ.get("MY_OPENAI_API_KEY")

def chain_maker(script_db):
    """
    script db에서 사용가능한 script를 찾아 답변하는 chain 생성

    Parameters:
        script_db : script들이 저장된 db
    Returns:
        rag chain
     """
    script_retriever = script_db.as_retriever(search_type="similarity", search_kwargs={"k": 1})
    prompt = PromptTemplate.from_template(
    """
    persona : story teller
    language : only korean
    tell dramatic story like talking to friend,
    speak informally,
    progress chapter by chapter,
    **hide header like '###'**,
    start chapter with interesting question,
    wait user answer
    give reaction to answer,
    do not use same reaction
    
    # script
    {script}

    #Previous Chat History:
    {chat_history}

    #Question: 
    {question} 
    """
    )

    llm = ChatOpenAI(model="gpt-4o-mini", api_key= openai.api_key, temperature=0.3)

    # 단계 8: 체인(Chain) 생성
    chain = (
        {
            "script" : itemgetter("question") | script_retriever,
            "question": itemgetter("question"),
            "chat_history": itemgetter("chat_history"),
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain

def history_chain(chain, memory_store : dict):
    """
    맥락을 유지하면 대화하는 chain 생성

    Parameters:
        chain : script 를 찾아 답변하는 chain
        memory_store : 대화의 맥락이 저장될 공간
    Returns:
        memory history chain
     """
    def get_session_history(session_ids):
        print(f"[대화 세션ID]: {session_ids}")
        if session_ids not in memory_store:  # 세션 ID가 store에 없는 경우
            # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
            memory_store[session_ids] = ChatMessageHistory()
        return memory_store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


    # 대화를 기록하는 RAG 체인 생성
    rag_with_history = RunnableWithMessageHistory(
        chain,
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
        history_messages_key="chat_history",  # 기록 메시지의 키
    )
    return rag_with_history

In [62]:
store ={}
chain = chain_maker(script_db)
h_chain = history_chain(chain, store)

while True:
      print("========================")
      query = input("어떤 이야기를 듣고 싶어? : ")
      if query.lower() == "exit":
            print("대화를 종료합니다.")
            break
      response = h_chain.invoke(
    # 질문 입력
    {"question": query},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "test18"}},
)
      print(query)
      print("\n답변:")
      print(response)

print(store['test18'])

[대화 세션ID]: test18
정안숙 사건

답변:
너, 정인숙 사건에 대해 들어본 적 있어? 이 사건은 정말 미스터리하고 복잡한 이야기야. 너는 어떤 부분이 제일 궁금해?
[대화 세션ID]: test18
전체 사건

답변:
정인숙 사건, 정말 흥미로운 주제지! 이 사건은 단순한 범죄 사건이 아니라, 한국 사회의 권력 구조와 정치적 음모가 얽힌 복잡한 이야기야. 그럼, 이 사건의 배경부터 시작해볼까? 

1970년대 초, 한국은 군사 정권 하에 있었고, 정치적 억압과 부패가 만연했어. 정인숙은 고위 공무원의 딸로 태어나서 고급 호스티스로 일하며 여러 정치인들과의 관계를 맺었지. 그녀의 죽음은 단순한 개인의 비극이 아니라, 그 시대의 사회적 맥락과 깊은 연관이 있어. 

너는 정인숙의 개인사에 대해 더 알고 싶어, 아니면 사건 발생 당시의 상황이 더 궁금해?
[대화 세션ID]: test18
당시의 상황

답변:
그럼 사건 발생 당시의 상황에 대해 이야기해볼게! 1970년 3월 17일, 서울은 군사 정권의 압박 속에 있었고, 사회는 정치적 억압과 부패가 만연했어. 그런 시기에 정인숙은 고급 호스티스로 일하면서 여러 정치인들과의 복잡한 관계를 맺고 있었지. 

그날 밤, 정인숙과 그녀의 넷째 오빠 정종욱은 마포구 합정동에서 총격을 당했어. 정인숙은 머리와 가슴에 총을 맞고 사망했고, 정종욱은 부상을 입었지. 처음에는 교통사고로 위장되었지만, 곧 살인 사건으로 밝혀졌어. 

이 사건은 단순한 범죄가 아니라, 그 시대의 권력자들의 비리와 음모가 얽힌 복잡한 이야기였어. 너는 이 사건이 어떻게 진행될지 궁금해, 아니면 정인숙의 개인사에 대해 더 알고 싶어?
[대화 세션ID]: test18
정안숙의 개인사는 어때

답변:
정인숙의 개인사? 그건 정말 흥미로운 부분이야! 정인숙은 고위 공무원의 딸로 태어나서, 대학을 중퇴한 후 고급 호스티스로 일하게 되었어. 그녀는 여러 정치인들과의 복잡한 관계를 맺으면서, 그 시대의 권력 구조 속에서 살아갔지.

특히 그녀는 3세 아들을